In [5]:
import pandas as pd

Read the data for January. How many columns are there?


In [6]:
# install pyarrow
# !pip install pyarrow

In [7]:
df=pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_val=pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [8]:
import os
print(os.getcwd())

/workspaces/mlops/week 1


In [9]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [10]:
len(df.columns)

19

Computing duration


In [11]:
df['duration']=df.tpep_dropoff_datetime-df.tpep_pickup_datetime
df.duration=df.duration.apply(lambda td: td.total_seconds() / 60)

In [12]:
df.duration.std()

np.float64(42.59435124195458)

Dropping outliers: 


In [13]:
og_len=len(df)
og_len

3066766

In [14]:
df_train = df[(df.duration >= 1) & (df.duration <= 60)]
new_len=len(df_train)

In [15]:
df_val['duration']=df_val.tpep_dropoff_datetime-df_val.tpep_pickup_datetime
df_val.duration=df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [16]:
new_len/og_len * 100

98.1220282212598

One-hot encoding:

In [17]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [18]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

df_train[categorical] = df_train[categorical].astype(str)

/tmp/ipykernel_7003/2190808870.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[categorical] = df_train[categorical].astype(str)


In [19]:
train_dicts = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [20]:
X_train.size

6018346

In [21]:
X_train.shape

(3009173, 515)

Training:

In [22]:
target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261932106969

Evaluating:

In [23]:
df_val[categorical] = df_val[categorical].astype(str)


In [24]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [25]:
len(val_dicts), len(train_dicts)

(2855951, 3009173)

In [26]:
X_val = dv.transform(val_dicts)

In [27]:
target = 'duration'
y_val = df_val[target].values

y_pred_val = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred_val)

7.811818743246608